# Evaluation of SIRET index on French RESTAURANTS open database

In [16]:
%reload_ext autoreload
%autoreload 2

In [17]:
from worksites import WorkSiteName, GeoWorkSiteName, geocode_worksites
from queries import request_elastic
import pandas as pd
import os
import elasticsearch as es

In [11]:
os.remove("export_alimconfiance.csv")
!wget -c https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72 -O export_alimconfiance.csv

--2022-11-24 14:51:26--  https://www.data.gouv.fr/fr/datasets/r/fff0cc27-977b-40d5-9c11-f7e4e79a0b72
Resolving www.data.gouv.fr (www.data.gouv.fr)... 37.59.183.93
Connecting to www.data.gouv.fr (www.data.gouv.fr)|37.59.183.93|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://dgal.opendatasoft.com/explore/dataset/export_alimconfiance/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true [following]
--2022-11-24 14:51:26--  https://dgal.opendatasoft.com/explore/dataset/export_alimconfiance/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true
Resolving dgal.opendatasoft.com (dgal.opendatasoft.com)... 34.249.199.226, 34.248.20.69
Connecting to dgal.opendatasoft.com (dgal.opendatasoft.com)|34.249.199.226|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/csv]
Saving to: ‘export_alimconfiance.csv’

export_alimconfianc     [          <=>       ]   6.54M   328KB/s    in 19s  

In [ ]:
# Preparing evaluatest dataset (database of French restaurants)

In [12]:
size_test = 1000
df = pd.read_csv("export_alimconfiance.csv", sep=";", dtype=str)[["APP_Libelle_etablissement", "SIRET", "Libelle_commune", "Code_postal", "Adresse_2_UA", "geores"]]

In [13]:
wks = [
    WorkSiteName(**{
        "siret":row["SIRET"],
        "name": row["APP_Libelle_etablissement"],
        "postcode": row["Code_postal"],
        "cityname": row["Libelle_commune"], 
        "address": row["Adresse_2_UA"]
    })
    for k, row in df.head(size_test).iterrows()
]

In [ ]:
geowks = geocode_worksites(wks)

In [ ]:
ELASTIC_URL = "http://elasticsearch-master:9200"
conn = es.Elasticsearch(ELASTIC_URL)

In [ ]:
# Check if the first response of Elasticsearch

In [ ]:
def first_matching_answer(conn, geoworksite, siret_truth:int, geo_threshold: float):
    output = request_elastic(conn, geowk, geo_threshold=geo_threshold)
    for ans in output:
        if ans["_source"]["siret"] == siret_truth:
            return ans
    return

def all_relevant_elastic_answers(conn, geoworksite, geo_threshold:float):
    output = request_elastic(conn, geowk, geo_threshold=geo_threshold)


score = 0
for geowk in geowks:
    answer = first_matching_answer(conn, geoworksite=geowk, siret_truth=geowk.siret, geo_threshold=geo_threshold)
    if answer:
        score += 1
    else:
        print("*****")
        print(geowk)
        output = request_elastic(conn, geowk)
        if output:
            first_response = output[0]["_source"]
            name = first_response.get("nom_commercial", None)
            try:
                print(f'{name} ({first_response["siret"]}) at {first_response["libelle_voie"]} {first_response["libelle_commune"]} {first_response["commune"]} - score: {output[0]["_score"]}')
            except:
                continue
        else:
            print("---")

print(f"SCORE OF RETRIEVAL: {score/size_test}%")

In [ ]:
#request_elastic(conn, geowks[0])